In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from tensorflow.keras import backend as K#
tf.compat.v1.set_random_seed(1)#
#sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
#K.set_session(sess)
import sys
#from tensorflow.keras import sequential

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten, Bidirectional
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.regularizers


import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import pickle
import collections
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TerminateOnNaN
from video_process_utils import *
import statsmodels.api as sm

In [22]:
target_col = 'GDI'

In [23]:
alldata_processed =\
    pd.read_csv("C:/Google Drive/PHD NEW/processed/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed['target_count'] = alldata_processed.groupby('videoid')[target_col].transform(lambda x: x.count())

In [24]:
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed[alldata_processed['target_count'] == 2]
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [25]:
datasplit_df = pd.read_csv('C:/Google Drive/PHD NEW/processed/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [26]:
with open('C:/Google Drive/PHD NEW/processed/all_processed_video_segments_doublesided.pickle', 'rb') as handle:
    processed_video_segments = pickle.load(handle)

In [27]:
assert(np.sum(alldata_processed[target_col].isnull()) == 0)
alldata_processed

,Patient_ID,examid,side,HipFlex_IC,HipRot_mean,KneeFlex_meanStance,KneeFlex_maxExtension,dxmod,dxside,faq,...,height_interpolated,mass_buckets,height_buckets,age_buckets,mass_interpolated2,age_truncated2,height_interpolated2,predicted_SEMLS,SEMLS_dev_residual,target_count
0,3466,6167,R,65.103000,42.098059,49.812311,43.406333,Diplegia,Bilateral,6,...,137.795,55.0,130.0,20.0,31.329767,4.000000,189.87462,0.085150,-0.421888,2
1,3466,6167,L,71.249667,-0.169098,52.859310,47.368000,Diplegia,Bilateral,6,...,137.795,55.0,130.0,20.0,31.329767,4.000000,189.87462,0.085150,-0.421888,2
2,3523,10742,R,44.691429,-14.877223,22.957506,9.374398,Hemiplegia,Left,10,...,164.000,65.0,160.0,20.0,47.196900,4.000000,268.96000,0.064930,-0.366425,2
3,3523,10742,L,47.951407,8.404332,17.942818,9.420285,Hemiplegia,Left,10,...,164.000,65.0,160.0,20.0,47.196900,4.000000,268.96000,0.064930,-0.366425,2
4,3561,8676,R,45.436579,-9.705642,29.875777,24.770209,Diplegia,Bilateral,0,...,175.000,65.0,170.0,20.0,48.580900,4.000000,306.25000,0.058696,-0.347818,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,10117,14849,L,43.525260,-3.446997,11.446938,7.087655,Diplegia,Bilateral,5,...,157.000,80.0,150.0,20.0,112.996900,4.000000,246.49000,0.043852,-0.299474,2
4420,10121,14857,R,33.502858,1.984192,17.656262,9.921491,Hemiplegia,Right,10,...,144.500,30.0,140.0,11.0,9.486400,1.431465,208.80250,0.202391,-0.672513,2
4421,10121,14857,L,42.464181,-0.131036,21.262660,9.046386,Hemiplegia,Right,10,...,144.500,30.0,140.0,11.0,9.486400,1.431465,208.80250,0.202391,-0.672513,2
4422,10162,14940,R,39.679867,-9.121854,17.941415,7.365353,Hemiplegia,Left,9,...,149.400,45.0,140.0,12.0,22.562500,1.616033,223.20360,0.178429,-0.626956,2


In [28]:
x_columns_left = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,50,52,54,56]
x_columns_right = [2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,51,53,55,57]

target_dict_L = {}
target_dict_R = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    if row['side'] == 'L':
        target_dict_L[row['videoid']] = row[target_col]
    if row['side'] == 'R':
        target_dict_R[row['videoid']] = row[target_col]

In [29]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)
X = np.concatenate([X[:,:,x_columns_left],X[:,:,x_columns_right]])

In [30]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)
X_train = np.concatenate([X_train[:,:,x_columns_left],X_train[:,:,x_columns_right]])

In [31]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)
X_validation = np.concatenate([X_validation[:,:,x_columns_left],X_validation[:,:,x_columns_right]])

In [32]:
y_train_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train = np.array(y_train_L + y_train_R)

In [33]:
y_validation_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation = np.array(y_validation_L + y_validation_R)

In [34]:
y_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in all_ids]
y_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in all_ids]
y = np.array(y_L + y_R)

In [35]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [36]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = train_videoid_weights + train_videoid_weights
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = validation_videoid_weights + validation_videoid_weights
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [37]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)
print(target_min, target_range)

32.5990731990764 80.74813196013362


In [38]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)
y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
#c_i_factor is just a constant initially introduced to make the loss function more interpretable, but
#is probably not necessary if the model were to be retrained
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights])) 

In [39]:
vid_length = 124

In [4]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#this is just for monitoring
mse_metric = w_mse(target_range**2*np.array(weights))

K.clear_session()

model = Sequential()
model.add(LSTM(128, input_shape=(vid_length,X_train.shape[2]), activation='tanh', return_sequences=True))

model.add(Bidirectional(LSTM(128,activation='tanh')))
# model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(BatchNormalization())
model.add(Flatten())

# model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()

NameError: name 'target_range' is not defined

In [76]:
checkpoint_folder = "C:/Google Drive/PHD NEW/checkpoints/rnn_Bidirect%s" % (target_col)
print(checkpoint_folder)

C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI


In [77]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)


    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_2', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=32, epochs=n_epochs,shuffle=True)

Epoch 1/100
1277/1277 [==============================] - ETA: 0s - loss: 144.6810
Epoch 00001: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-01-102.7276.hdf5
1277/1277 [==============================] - 157s 123ms/step - loss: 144.6810 - val_loss: 102.7276 - lr: 0.0010
Epoch 2/100
1277/1277 [==============================] - ETA: 0s - loss: 100.4474
Epoch 00002: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-02-107.0589.hdf5
1277/1277 [==============================] - 163s 128ms/step - loss: 100.4474 - val_loss: 107.0589 - lr: 0.0010
Epoch 3/100
1277/1277 [==============================] - ETA: 0s - loss: 90.6923
Epoch 00003: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-03-93.1357.hdf5
1277/1277 [==============================] - 161s 126ms/step - loss: 90.6923 - val_loss: 93.1357 - lr: 0.0010
Epoch 4/100
1277/1277 [==============================] - ETA: 0s - loss: 86.7760
Epoch 00004: saving mod

Epoch 28/100
1277/1277 [==============================] - ETA: 0s - loss: 45.4088
Epoch 00028: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-28-82.5024.hdf5
1277/1277 [==============================] - 252s 198ms/step - loss: 45.4088 - val_loss: 82.5024 - lr: 6.4000e-04
Epoch 29/100
1277/1277 [==============================] - ETA: 0s - loss: 44.2344
Epoch 00029: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-29-81.9022.hdf5
1277/1277 [==============================] - 252s 197ms/step - loss: 44.2344 - val_loss: 81.9022 - lr: 6.4000e-04
Epoch 30/100
1277/1277 [==============================] - ETA: 0s - loss: 40.9546
Epoch 00030: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-30-86.0108.hdf5
1277/1277 [==============================] - 250s 196ms/step - loss: 40.9546 - val_loss: 86.0108 - lr: 5.1200e-04
Epoch 31/100
1277/1277 [==============================] - ETA: 0s - loss: 39.4608
Epoch 00031: sa

1277/1277 [==============================] - ETA: 0s - loss: 8.0021
Epoch 00082: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-82-91.4699.hdf5
1277/1277 [==============================] - 251s 197ms/step - loss: 8.0021 - val_loss: 91.4699 - lr: 1.6777e-04
Epoch 83/100
1277/1277 [==============================] - ETA: 0s - loss: 7.8100
Epoch 00083: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-83-94.7446.hdf5
1277/1277 [==============================] - 249s 195ms/step - loss: 7.8100 - val_loss: 94.7446 - lr: 1.6777e-04
Epoch 84/100
1277/1277 [==============================] - ETA: 0s - loss: 7.6508
Epoch 00084: saving model to C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-84-93.4673.hdf5
1277/1277 [==============================] - 250s 196ms/step - loss: 7.6508 - val_loss: 93.4673 - lr: 1.6777e-04
Epoch 85/100
1277/1277 [==============================] - ETA: 0s - loss: 7.4944
Epoch 00085: saving model to C:/Goo

In [78]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [79]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename']
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [80]:
#todo: change so we also have a X,y for all_ids
#then for each of 100 epochs, generate the predictions of the model for each epoch
#this dataframe can then be saved, and a file called "get_best_epoch" can be used to generate the best one
#(video_id,target,predictions,dataset) 

In [81]:
def predict_and_aggregate(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    side_col = ['L' if i < len(df)/2 else 'R' for i in range(len(df))]
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    videoids = videoids + videoids
    df["videoid"] = np.array(videoids)
    df['side'] = side_col
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid','side'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [82]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
video_ids = video_ids + video_ids
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
side_col = ['L' if i < len(predictions_df)/2 else 'R' for i in range(len(predictions_df))]
predictions_df[target_col] = y
predictions_df['side'] = side_col
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [83]:
for i in range(1,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-01-102.7276.hdf5
weights-02-107.0589.hdf5
weights-03-93.1357.hdf5
weights-04-80.7612.hdf5
weights-05-85.3685.hdf5
weights-06-86.2160.hdf5
weights-07-79.5788.hdf5
weights-08-77.7051.hdf5
weights-09-76.7576.hdf5
weights-10-94.7455.hdf5
weights-100-94.4004.hdf5
weights-11-81.4550.hdf5
weights-12-73.0268.hdf5
weights-13-76.2135.hdf5
weights-14-79.1429.hdf5
weights-15-76.1950.hdf5
weights-16-79.3493.hdf5
weights-17-81.1037.hdf5
weights-18-80.0472.hdf5
weights-19-75.0462.hdf5
weights-20-75.7465.hdf5
weights-21-78.6205.hdf5
weights-22-80.0585.hdf5
weights-23-79.1198.hdf5
weights-24-79.1048.hdf5
weights-25-78.9801.hdf5
weights-26-82.4880.hdf5
weights-27-79.1871.hdf5
weights-28-82.5024.hdf5
weights-29-81.9022.hdf5
weights-30-86.0108.hdf5
weights-31-82.5200.hdf5
weights-32-83.5013.hdf5
weights-33-81.6572.hdf5
weights-34-86.3182.hdf5
weights-35-86.1478.hdf5
weights-36-85.7343.hdf5
weights-37-87.0718.hdf5
weights-38-83.1552.hdf5
weights-39-85.8862.hdf5
weights-40-90.2056.hdf5
weights-41-84

<ipython-input-83-c58657afd95e>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)


weights-97-93.8310.hdf5
weights-98-95.7786.hdf5
weights-99-96.2240.hdf5


In [84]:
predictions_df.groupby(['videoid','side','dataset'],as_index=False).mean().to_csv("C:/Google Drive/PHD NEW/predictions_new/rnn_bi_%s_predictions_all_epochs.csv" % (target_col),index=False)

In [85]:
# Save best models
maps = {
    "GDI": "C:/Google Drive/PHD NEW/checkpoints/rnn_BidirectGDI/weights-19-75.0462.hdf5",
}
for col in maps.keys():
    model_folder_path = "C:/Google Drive/PHD NEW/models/%s_best_new_bi.pb" % (col)
    model.load_weights(maps[col])
    model.save(model_folder_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: C:/Google Drive/PHD NEW/models/GDI_best_new_bi.pb\assets
